In [2]:
import gurobipy as grb
import pandas as pd

mod = grb.Model()

FC = centers.index
FC=FC.tolist()
item=items.index.tolist()
region = regions.index.tolist()


inputFile='data.xlsx'
data=pd.read_excel(inputFile,sheet_name=None,index_col=0)
centers=data['Fulfilment Centers']
regions=data['Regions']
distances=data['Distances']
items=data['Items']
demand=data['Demand']
weight = pd.Series(items.iloc[:,0].tolist(),index=item)
size=pd.Series(items.iloc[:,1].tolist(),index=item)
capacity = pd.Series(centers.iloc[:,1].tolist(),index=FC)
distance = pd.DataFrame(distances,index=region,columns=FC)
demand = pd.DataFrame(demand,index = item,columns = region)

In [3]:
x={}
for i in item:
    for j in FC:
        for k in region:
            x[i,j,k] = mod.addVar(lb=0,name='x[{0},{1},{2}]'.format(i,j,k))


In [4]:
mod.setObjective(1.38*sum(weight[i] *sum(x[i,j,k]*distance.loc[k,j] for j in FC for k in region) for i in item),sense=grb.GRB.MINIMIZE)

In [5]:
cap = {}
for j in FC:
    cap[j]=mod.addConstr(sum(x[i,j,k]*size.loc[i] for i in item for k in region)<=capacity.loc[j],name='Capacity {0}'.format(j))
for i in item:
    for k in region:
        mod.addConstr(sum(x[i,j,k] for j in FC)>=demand.loc[i,k],name='Demand {0} {1}'.format(i,k))

In [6]:
##run
mod.setParam('OutputFlag',False)
mod.optimize()

In [7]:
##print outputs
print('Optimal Objective Value: {0:.2f}'.format(mod.ObjVal))
shipment = []
for i in item:
    for j in FC:
        for k in region:
            shipment.append([j,k,i,x[i,j,k].x])
shipmentDF = pd.DataFrame(shipment,columns=['FC','Region','Item','Quantity'])
shipmentDF.to_excel('shipment.xlsx')

output = []
for j in cap:
    output.append([j,cap[j].PI])
outDF = pd.DataFrame(output,columns = ['FC','Shadow Price'])
outDF.to_excel('output.xlsx')

Optimal Objective Value: 9841229.29
